# Visualization and trajectory analysis of AAXAA MD simulation

As always, our first step is loading in the libraries that we need and then reading in our MD trajectory file:

In [ ]:
import mdtraj as md
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format='retina'

In [ ]:
traj = md.load('YYYYY_sim.dcd', top='YYYYY.prmtop') # replace YYYYY with your peptide's name

In [ ]:
# OPTIONAL: visualize your trajectory (note: won't work in Jupyter Lab!)
import nglview as nv
view = nv.show_mdtraj(traj)
view.clear_representations()
view.add_licorice()
view

One thing we want to do is calculate the end-to-end distance of the peptide at each frame (or snapshot) of the trajectory. To do this, we need to identify a specific pair of atoms that can serve as the "ends" of the molecule. Let's choose the C$_\alpha$ atoms of the first residue (1) and last residue (5):

In [ ]:
atoms, bonds = traj.topology.to_dataframe()
atoms[atoms['name'] == 'CA']

Once you have the correct atom indices, insert them into the code below:

In [ ]:
atom_indices = [] # list of two CA atoms that represent opposite ends of molecule
etoe = md.compute_distances(traj, [atom_indices])
print(etoe.shape)

plt.hist(etoe)
plt.xlabel('End-to-end distance (nm)')
plt.ylabel('Counts')
plt.show()

The other quantity that we'll compute is the solvent-accessible surface area (or polar surface area) of the molecule:

In [ ]:
sasa = md.shrake_rupley(traj)
sasa.shape

You might notice that there are *a lot* of columns in these data. This is because the solvent-accessible surface area calculation outputs this quantity for each atom in the molecule. We'll collapse this down into a single number for the entire molecule below:

In [ ]:
sasa_whole = sasa.sum(axis=1) # this sums up all the columns in each row of sasa
sasa_whole = sasa_whole.reshape((len(sasa), 1))
print(sasa_whole.shape)

plt.hist(sasa_whole)
plt.xlabel('Solvent-accessible surface area (nm$^2$)')
plt.show()

You might be interested to see a single 2D histogram that takes into account both quantities:

In [ ]:
plt.figure(figsize=(7.5,6))
plt.hist2d(etoe.flatten(), sasa_whole.flatten(), bins=(25,25), 
           cmin=1, cmap='plasma')
plt.xlabel('End-to-end distance (nm)')
plt.ylabel('Solvent-accessible surface area (nm$^2$)')
plt.colorbar()
plt.show()

# Clustering the trajectory analysis data

One common task is to group all the different structures generated by the simulation into a small number of clusters and then see which characteristics are common to each cluster. In what follows, we'll focus just on clustering the structures from our trajectory using two different algorithms:

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import minmax_scale

X = np.concatenate((etoe, sasa_whole), axis=1) # construct the input data X
X.shape # check that the shape is correct: (n_samples, n_features)

As discussed in our previous clustering exercise, it's often good to scale the data if the ranges of the features are very different from each other. In this case, the range of the two features are *not* that different, but let's do the min-max scaling nonetheless:

In [ ]:
X2 = minmax_scale(X)
X2

As a first pass, we can try the *k*-means algorithm that we used before. We'll start with 3 clusters, but feel free to experiment with more (or fewer):

In [ ]:
KMclusters = KMeans(n_clusters=3, random_state=0).fit(X2)

plt.figure(figsize=(6,6))
plt.scatter(etoe, sasa_whole, c=KMclusters.labels_, 
            cmap='tab10', alpha=0.5)
plt.xlabel('End-to-end distance (nm)')
plt.ylabel('Solvent-accessible surface area (nm$^2$)')
plt.show()

Next, we'll try the Gaussian mixture algorithm... 

The terminology used for the arguments of the `GaussianMixture()` function is somewhat different than those of the `KMeans()` function. One big difference is that `KMeans()` uses `n_clusters`, whereas `GaussianMixture()` uses `n_components`. Also, in order to generate cluster labels using `GaussianMixture()` we need to use `.fit_predict()` rather than `.fit()`.

In [ ]:
GMclusters = GaussianMixture(n_components=3, random_state=0, n_init=50).fit_predict(X2)

plt.figure(figsize=(6,6))
plt.scatter(etoe, sasa_whole, c=GMclusters, 
            cmap='tab10', alpha=0.5)
plt.xlabel('End-to-end distance (nm)')
plt.ylabel('Solvent-accessible surface area (nm$^2$)')
plt.show()

## Your turn: evaluating the clustering results

Clearly the *k*-means algorithm gives different results from the Gaussian mixture algorithm. Based on the 2D histogram above, which clustering algorithm do you think gives more meaningful results?